In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, accuracy_score, f1_score, matthews_corrcoef
from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import TensorDataset, DataLoader, random_split, ConcatDataset, Subset, Dataset

In [5]:
def metrics_output(preds,labels):
    true_labels = np.array(labels)
    predicted_probs = np.array(preds)
    binary_predictions = (predicted_probs >= 0.5).astype(int)
    auc = roc_auc_score(true_labels, predicted_probs)
    conf_matrix = confusion_matrix(true_labels, binary_predictions)
    tn, fp, fn, tp = conf_matrix.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy = accuracy_score(true_labels, binary_predictions)
    f1 = f1_score(true_labels, binary_predictions)
    mcc = matthews_corrcoef(true_labels, binary_predictions)  
    return (auc, sensitivity, specificity, accuracy, f1, mcc)

In [6]:
class TransformerEncoderClassification(nn.Module):
    def __init__(self):
        super(TransformerEncoderClassification, self).__init__()
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=8, nhead=4), #8 确定，4定为能使d_model divisible by nhead 
            num_layers= 3,
        ) 
        self.fc = nn.Linear(24, 2)# 4 batch, 24 samples every batch

    def forward(self, x):
        x = x.permute(1, 0, 2)  
        x = self.transformer_encoder(x) 
        x = x.permute(1, 0, 2) 
        x = x.flatten(1) 
        print(x.shape)
        x = self.fc(x)
        return x
    
model = TransformerEncoderClassification() 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.005)

/Users/jiaming/Desktop/yes/envs/torch_nightly_env/lib/python3.8/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [7]:
loaded_datasets_info = torch.load('/Users/jiaming/Desktop/Lab2/datas/saved_datasets_scaled.pth')
loaded_train_dataset = loaded_datasets_info['train_dataset'] # all data, can use batch to split into batches
loaded_val_dataset = loaded_datasets_info['val_dataset']
loaded_test_dataset = loaded_datasets_info['test_dataset']

In [8]:
# split x and y from loader
X_train = []
y_train = []
for x, y in loaded_train_dataset:
    X_train.append(x.reshape(-1, 3, 8))
    y_train.append(y)
    
X_train = torch.cat(X_train, dim=0)
y_train = torch.cat(y_train, dim=0) 

print(X_train.shape) # 824 samples
print(y_train.shape)

############################################

X_val = []
y_val = []
for x, y in loaded_val_dataset:
    X_val.append(x.reshape(-1, 3, 8))
    y_val.append(y)
    
X_val = torch.cat(X_val, dim=0)
y_val = torch.cat(y_val, dim=0) 

print(X_val.shape) # 206 samples
print(y_val.shape)

############################################

X_test = []
y_test = []
for x, y in loaded_test_dataset:
    X_test.append(x.reshape(-1, 3, 8))
    y_test.append(y)

X_test = torch.cat(X_test, dim=0)
y_test = torch.cat(y_test, dim=0) 

print(X_test.shape) # 258 samples
print(y_test.shape)



torch.Size([824, 3, 8])
torch.Size([824])
torch.Size([206, 3, 8])
torch.Size([206])
torch.Size([258, 3, 8])
torch.Size([258])


In [9]:
# package up data and label
class MyDataSet(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label
        self.length = label.shape[0]
        
    def __getitem__(self, index):
        return self.data[index], self.label[index]
    def __len__(self):
        return self.length

In [10]:
train_dataset = MyDataSet(X_train, y_train) 
test_dataset = MyDataSet(X_test, y_test) 
val_dataset = MyDataSet(X_val, y_val) 

In [11]:
from sklearn.model_selection import KFold

# Example dataset
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([0, 1, 0, 1, 0, 1])

# Initialize KFold
kf = KFold(n_splits=3)

# Splitting the dataset
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


TRAIN: [2 3 4 5] TEST: [0 1]
TRAIN: [0 1 4 5] TEST: [2 3]
TRAIN: [0 1 2 3] TEST: [4 5]


In [12]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
train_dataset = ConcatDataset([train_dataset, val_dataset]) # 824(train)+206(val)=1030
train_loader = DataLoader(train_dataset, batch_size=103) # 103样本每批，总用10批

In [13]:
X_tpv = []
y_tpv = []
for x, y in train_dataset:
    X_tpv.append(x)
    y_tpv.append(y)

print(len(X_tpv)) # 1030 samples
print(len(y_tpv))

1030
1030


In [14]:
X_tpvl = []
y_tpvl = []
for x, y in train_loader.dataset:
    X_tpvl.append(x)
    y_tpvl.append(y)

print(len(X_tpvl)) # 1030 samples
print(len(y_tpvl))

1030
1030


In [15]:
for fold_idx, (train_index, val_index) in enumerate(kf.split(train_dataset)):
    print(fold_idx, (train_index, val_index))
    predicted_probabilities = []
    true_labels = []
    predicted_labels = []  
    
    train_loader_fold = DataLoader(Subset(train_dataset, train_index), batch_size=4) #824 #206/batch 4batch
    val_loader_fold = DataLoader(Subset(train_dataset, val_index), batch_size=1) #206 1batch
    
    X_tpvl = []
    y_tpvl = []
    for x, y in train_loader_fold:
        X_tpvl.append(x)
        y_tpvl.append(y)
    print(len(X_tpvl)) # 1030 samples
    print(len(y_tpvl))
    
    for epoch in range(15):    
        model = TransformerEncoderClassification() 
        for batch_idx, (features, targets) in enumerate(train_loader_fold):     
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, targets.long()) 
            loss.backward()
            optimizer.step()
            
        with torch.set_grad_enabled(False):
            for batch_idx, (features, targets) in enumerate(val_loader_fold):
                logits = model(features.float())
                _, y_pred = torch.max(logits.data, dim=1)
                predicted_labels.extend(y_pred.tolist()) 
                logits = torch.nn.functional.softmax(logits)
                predicted_probabilities.extend(logits[:,1].tolist()) 
                true_labels.extend(targets.tolist())
        #torch.save(model.state_dict(), f'D:/Jupyter_directory/Res-F5C-main/Results(Yue)/Geo/VIT/transformer_{fold_idx}.pth')
        #np.save(f'D:/Jupyter_directory/Res-F5C-main/Results(Yue)/Geo/VIT/y_val_pred_{fold_idx}.npy', predicted_probabilities)
        #np.save(f'D:/Jupyter_directory/Res-F5C-main/Results(Yue)/Geo/VIT/y_val_{fold_idx}.npy', true_labels)

0 (array([   0,    1,    4,    5,    6,    7,    8,    9,   11,   12,   13,
         14,   15,   16,   17,   18,   19,   20,   21,   22,   24,   26,
         27,   28,   29,   32,   33,   34,   35,   36,   37,   38,   40,
         41,   42,   43,   45,   46,   47,   48,   49,   50,   51,   52,
         53,   56,   57,   58,   61,   62,   64,   65,   68,   69,   71,
         73,   74,   75,   77,   78,   79,   80,   81,   82,   83,   84,
         85,   87,   89,   90,   91,   93,   94,   95,   97,   98,   99,
        102,  103,  104,  105,  108,  111,  112,  113,  114,  115,  116,
        117,  118,  119,  121,  122,  123,  124,  125,  126,  127,  128,
        129,  130,  131,  132,  133,  134,  135,  137,  138,  140,  141,
        142,  143,  144,  145,  146,  147,  148,  149,  150,  151,  152,
        153,  154,  155,  156,  157,  159,  160,  161,  162,  163,  164,
        165,  166,  167,  169,  170,  171,  172,  173,  175,  176,  177,
        178,  179,  180,  181,  182,  183,  185,

/Users/jiaming/Desktop/yes/envs/torch_nightly_env/lib/python3.8/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])


/var/folders/yy/glnpb5vs3n3b2fw9ncjqg4b80000gn/T/ipykernel_86252/431107765.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits = torch.nn.functional.softmax(logits)


torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])


...

torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])


torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])


torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])


torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])


torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([4, 24])
torch.Size([1, 24])
torch.Size([1, 24])
torch.Size([1, 24])
torch.Size([1, 24])
torch.Size([1, 24])
torch.Size([1, 24])
torch.Size([1, 24])
torch.Size([1, 24])
torch.Size([1, 24])
torch.Size([1, 24])
torch.Size([1, 24])
torch.Size([1, 24])
torch.Size([1, 24])
torch.Size([1, 24])
torch.Size([1, 24])


In [16]:
#model = Net_conv(input_length = datas.shape[2])
#model.load_state_dict(torch.load('D:/Jupyter_directory/Res-F5C-main/Results(Yue)/Geo/VIT/transformer_{fold_idx}.pth'))
#predicted_probabilities = np.load(f'D:/Jupyter_directory/Res-F5C-main/Results(Yue)/Geo/VIT/y_val_pred_{fold_idx}.npy')
#true_labels = np.load(f'D:/Jupyter_directory/Res-F5C-main/Results(Yue)/Geo/VIT/y_val_{fold_idx}.npy')

In [17]:
roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC = metrics_output(predicted_probabilities,true_labels)
print(roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC)

0.47373645754598137 0.5142857142857142 0.4567901234567901 0.4841423948220065 0.4867997424339987 -0.028940305470626086


In [249]:
#np.save('D:/Jupyter_directory/Res-F5C-main/Results(Yue)/Geo/VIT/y_val_pred.npy', predicted_probabilities)
#np.save('D:/Jupyter_directory/Res-F5C-main/Results(Yue)/Geo/VIT/y_val.npy', true_labels)

In [250]:
predicted_labels = []  
true_labels = [] 
predicted_probabilities = []
with torch.set_grad_enabled(False): 
    for batch_idx, (features, targets) in enumerate(test_dataloader):
        logits = model(features.float())
        _, y_pred = torch.max(logits.data, dim=1)
        predicted_labels.extend(y_pred.tolist()) 
        logits = torch.nn.functional.softmax(logits)
        predicted_probabilities.extend(logits[:,1].tolist()) 
        true_labels.extend(targets.tolist())

torch.Size([258, 24])


/var/folders/yy/glnpb5vs3n3b2fw9ncjqg4b80000gn/T/ipykernel_82752/1434766837.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits = torch.nn.functional.softmax(logits)


In [251]:
roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC = metrics_output(predicted_probabilities,true_labels)
print(roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC)

0.5673400673400674 0.5075757575757576 0.5793650793650794 0.5426356589147286 0.5317460317460317 0.08712963376890503


In [18]:
#np.save('D:/Jupyter_directory/Res-F5C-main/Results(Yue)/Geo/VIT/y_test_pred.npy', predicted_probabilities)
#np.save('D:/Jupyter_directory/Res-F5C-main/Results(Yue)/Geo/VIT/y_test.npy', true_labels)

In [19]:
# train_dataset
X_t = []
y_t = []
for x, y in train_dataset:
    X_t.append(x)
    y_t.append(y)
print(len(X_t)) # 1030 samples
print(len(y_t))

1030
1030


In [21]:
train_loader = DataLoader(train_dataset, batch_size=515) #每个batch内有515个sample, 2个batch

X_tl = []
y_tl = []
for x, y in train_loader:
    X_tl.append(x)
    print(X_tl) 
    y_tl.append(y)
print(len(X_tl)) # 2批
print(len(y_tl)) # 2批

for batch_idx, (x, y) in enumerate(train_loader):
    print(batch_idx) # 0-1
    X_tl.append(x)
    y_tl.append(y)
    #print(len(X_tl) #look at the final 
    #print(len(y_tl) #look at the final 

[tensor([[[-1.4230, -0.8022,  2.2619,  ..., -0.7858, -0.7142, -0.2119],
         [ 0.9952, -0.9952, -0.9526,  ..., -0.3256,  1.0346, -1.0346],
         [ 0.2106, -0.2106,  0.0926,  ..., -0.7666,  0.1841, -0.1841]],

        [[-0.2656, -0.8022,  0.6425,  ...,  0.4894, -0.5452, -0.2878],
         [-1.0049,  1.0049, -0.9526,  ..., -0.3256,  1.0346, -1.0346],
         [ 0.2106, -0.2106,  0.0926,  ...,  1.3045,  0.1841, -0.1841]],

        [[-1.1915,  0.8086,  0.3649,  ..., -0.2015,  0.3001, -0.2499],
         [-1.0049,  1.0049, -0.9526,  ..., -0.3256, -0.9666,  0.9666],
         [ 0.2106, -0.2106,  0.0926,  ..., -0.7666,  0.1841, -0.1841]],

        ...,

        [[ 0.7760,  0.8086,  0.3649,  ...,  0.2809, -0.1226, -0.3637],
         [-1.0049,  1.0049,  1.0498,  ..., -0.3256, -0.9666,  0.9666],
         [ 0.2106, -0.2106,  0.0926,  ...,  1.3045,  0.1841, -0.1841]],

        [[ 1.8177, -0.8022,  0.5500,  ...,  0.9039, -0.1226, -0.1740],
         [ 0.9952, -0.9952,  1.0498,  ..., -0.3256,  1

In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Create some dummy data and labels
data = torch.randn(100, 10)  # 100 samples, 10 features each
labels = torch.randint(0, 2, (100,))  # 100 labels (binary classification)

# Create Dataset
dataset = TensorDataset(data, labels)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Iterate over DataLoader
for batch_idx, (data, labels) in enumerate(dataloader):
    print(f"Batch {batch_idx + 1}: {data.shape[0]} samples")


Batch 1: 2 samples
Batch 2: 2 samples
Batch 3: 2 samples
Batch 4: 2 samples
Batch 5: 2 samples
Batch 6: 2 samples
Batch 7: 2 samples
Batch 8: 2 samples
Batch 9: 2 samples
Batch 10: 2 samples
Batch 11: 2 samples
Batch 12: 2 samples
Batch 13: 2 samples
Batch 14: 2 samples
Batch 15: 2 samples
Batch 16: 2 samples
Batch 17: 2 samples
Batch 18: 2 samples
Batch 19: 2 samples
Batch 20: 2 samples
Batch 21: 2 samples
Batch 22: 2 samples
Batch 23: 2 samples
Batch 24: 2 samples
Batch 25: 2 samples
Batch 26: 2 samples
Batch 27: 2 samples
Batch 28: 2 samples
Batch 29: 2 samples
Batch 30: 2 samples
Batch 31: 2 samples
Batch 32: 2 samples
Batch 33: 2 samples
Batch 34: 2 samples
Batch 35: 2 samples
Batch 36: 2 samples
Batch 37: 2 samples
Batch 38: 2 samples
Batch 39: 2 samples
Batch 40: 2 samples
Batch 41: 2 samples
Batch 42: 2 samples
Batch 43: 2 samples
Batch 44: 2 samples
Batch 45: 2 samples
Batch 46: 2 samples
Batch 47: 2 samples
Batch 48: 2 samples
Batch 49: 2 samples
Batch 50: 2 samples


In [ ]:
for fold_idx, (train_index, val_index) in enumerate(train_loader.dataset):
    print()

In [1]:
import torch
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self):
        self.data = torch.randn(100, 10)  # 随机生成100个样本，每个样本10个特征
        self.labels = torch.randint(0, 2, (100,))  # 生成100个二分类标签

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

    def get_label_distribution(self):
        # 计算并返回标签分布
        return self.labels.float().mean()

# 创建数据集和数据加载器
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# 使用 dataloader.dataset
print("Dataset size:", len(dataloader.dataset))  # 打印数据集大小 #相当于不考虑batch划分，是原始dataset的大小
print("Label distribution:", dataloader.dataset.get_label_distribution())  # 调用自定义方法计算标签分布

# 访问第一个数据元素
first_data, first_label = dataloader.dataset[0]
print("First data sample:", first_data)
print("First label:", first_label)


Dataset size: 100
Label distribution: tensor(0.5000)
First data sample: tensor([ 0.1704, -0.4465, -0.0962, -0.2065, -1.0076,  0.2095, -1.5257, -0.1872,
         1.5675, -1.3254])
First label: tensor(1)
